## Data Downloading

In [1]:
# @title Kaggle Competition Data Downloading  { display-mode: "form",run :"auto" }
# @markdown Put the competition name
kaggle_username = 'Kaggle_Username'     # @param ['Kaggle_Username', 'kaggle_username', 'kaggle_2']
kaggle_token = "Kaggle"     # @param ["Kaggle", 'kaggle', 'kaggle_2_pass']

project_name = 'playground-series-s4e11'  # @param {type: "string"}


from google.colab import userdata
from IPython import display
import os
token = {"username":userdata.get(kaggle_username),"key":userdata.get(kaggle_token)}
import os, json
os.environ['KAGGLE_CONFIG_DIR']='.'
with open('kaggle.json', "w") as f:
    json.dump(token, f)
!chmod 600 ./kaggle.json
!kaggle competitions download -c $project_name
filename = project_name + ".zip"
!unzip $project_name && rm $filename
os.environ['MLFLOW_TRACKING_PASSWORD'] = userdata.get('MLFLOW_TRACKING_PASSWORD')
os.environ['MLFLOW_TRACKING_USERNAME'] = userdata.get('MLFLOW_TRACKING_USERNAME')
display.clear_output()
print("Files have been downloaded!")

Files have been downloaded!


In [2]:
!kaggle datasets download sumansharmadataworld/depression-surveydataset-for-analysis && unzip *.zip && rm -d *.zip

Dataset URL: https://www.kaggle.com/datasets/sumansharmadataworld/depression-surveydataset-for-analysis
License(s): CC0-1.0
  0% 0.00/42.6k [00:00<?, ?B/s]
100% 42.6k/42.6k [00:00<00:00, 51.4MB/s]
Archive:  depression-surveydataset-for-analysis.zip
  inflating: final_depression_dataset_1.csv  


In [3]:
!pip install optuna-dashboard optuna
!pip install mlflow dagshub
!pip install catboost
!mkdir -p output/models  output/logs output/plots
!pip install --upgrade gdown
display.clear_output()
import dagshub
import mlflow
import optuna

In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import anderson,kstest, shapiro
from IPython.display import Audio,display as dis
from IPython.display import clear_output

from sklearn.preprocessing import MinMaxScaler, RobustScaler, StandardScaler
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import LinearRegression, Ridge

In [5]:
# Classification
from sklearn.metrics import (confusion_matrix, accuracy_score,
                             precision_score, f1_score, recall_score,
                             roc_auc_score, roc_curve, auc
                             )
from sklearn.linear_model import LogisticRegression
from sklearn.dummy import DummyClassifier

from sklearn.neighbors import KNeighborsClassifier
from sklearn.neighbors import VALID_METRICS

from sklearn.tree import DecisionTreeClassifier, ExtraTreeClassifier
from sklearn.ensemble import (AdaBoostClassifier, ExtraTreesClassifier,
                              BaggingClassifier, RandomForestClassifier,
                              GradientBoostingClassifier,
                              HistGradientBoostingClassifier,
                              StackingClassifier
                              )
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier, XGBRFClassifier
from catboost import CatBoostClassifier

# Regression Models
from sklearn.metrics import (root_mean_squared_log_error as RMLE,
                             r2_score)
# 'neg_root_mean_squared_log_error'
from sklearn.model_selection import train_test_split

from sklearn.linear_model import LinearRegression

from sklearn.neighbors import KNeighborsRegressor
from sklearn.neighbors import VALID_METRICS

from sklearn.tree import DecisionTreeRegressor, ExtraTreeRegressor
from sklearn.ensemble import (AdaBoostRegressor, ExtraTreesRegressor,
                              BaggingRegressor, RandomForestRegressor,
                              GradientBoostingRegressor,
                              HistGradientBoostingRegressor,
                              StackingRegressor
                              )
from lightgbm import LGBMRegressor
from xgboost import XGBRegressor, XGBRFRegressor
from catboost import CatBoostRegressor

/usr/local/lib/python3.10/dist-packages/dask/dataframe/__init__.py:42: FutureWarning: 
Dask dataframe query planning is disabled because dask-expr is not installed.

You can install it with `pip install dask[dataframe]` or `conda install dask`.
This will raise in a future version.

  warnings.warn(msg, FutureWarning)


In [6]:
# plotly libraries setup
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objs as go
import plotly.io as pio
pio.renderers.default = "colab"
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)
# import plotly.offline as pyo
# pyo.init_notebook_mode()

In [8]:
dagshub.init("Kaggle-Competitions-Lab", "SantanuK", mlflow=True)
experiment_name = 'Exploring Mental Health Data'
try:
    mlflow.create_experiment(experiment_name)
    mlflow.set_experiment(experiment_name)
except Exception as e:
    mlflow.set_experiment(experiment_name)
    # print("Experiment has been created or some issue occured!\n","Error: ", e )

❗❗❗ AUTHORIZATION REQUIRED ❗❗❗

Output()



Open the following link in your browser to authorize the client:
https://dagshub.com/login/oauth/authorize?state=b8b08df2-5497-4b98-9d61-761a9b06c464&client_id=32b60ba385aa7cecf24046d8195a71c07dd345d9657977863b52e7748e0f0f28&middleman_request_id=383f0369626c84e16ceab954bbba25996d0eebb939083de10f90430db256804a




Accessing as SantanuK

Initialized MLflow to track repo "SantanuK/Kaggle-Competitions-Lab"

Repository SantanuK/Kaggle-Competitions-Lab initialized!

In [10]:
df_train = pd.read_csv('train.csv').drop('id',axis=1)
df_test = pd.read_csv('test.csv').drop('id',axis=1)
df_extra = pd.read_csv('final_depression_dataset_1.csv')

In [10]:
print(f"""The number of features: {df_test.shape[1]}
-----------------
Training DataSet
-----------------
The number of samples:              {df_train.shape[0]}
The number of duplicated samples:   {df_train.duplicated().sum()}
The number of null samples:         {df_train.isna().sum().sum()} ({round(df_train.isna().sum().sum()/df_train.shape[0]*100,2)}%)
The number of unique samples:       {df_train.nunique().sum()}

-----------------
Testing DataSet
-----------------
The number of samples:              {df_test.shape[0]}
The number of duplicated samples:   {df_test.duplicated().sum()}
The number of null samples:         {df_test.isna().sum().sum()}({round(df_test.isna().sum().sum()/df_test.shape[0]*100,2)}%)
The number of unique samples:       {df_test.nunique().sum()}

""")

The number of features: 18
-----------------
Training DataSet
-----------------
The number of samples:              140700
The number of duplicated samples:   0
The number of null samples:         430876 (306.24%)
The number of unique samples:       1180

-----------------
Testing DataSet
-----------------
The number of samples:              93800
The number of duplicated samples:   0
The number of null samples:         287291(306.28%)
The number of unique samples:       1062




In [9]:
df_train.head()

,Name,Gender,Age,City,Working Professional or Student,Profession,Academic Pressure,Work Pressure,CGPA,Study Satisfaction,Job Satisfaction,Sleep Duration,Dietary Habits,Degree,Have you ever had suicidal thoughts ?,Work/Study Hours,Financial Stress,Family History of Mental Illness,Depression
0,Aaradhya,Female,49.0,Ludhiana,Working Professional,Chef,NaN,5.0,NaN,NaN,2.0,More than 8 hours,Healthy,BHM,No,1.0,2.0,No,0
1,Vivan,Male,26.0,Varanasi,Working Professional,Teacher,NaN,4.0,NaN,NaN,3.0,Less than 5 hours,Unhealthy,LLB,Yes,7.0,3.0,No,1
2,Yuvraj,Male,33.0,Visakhapatnam,Student,NaN,5.0,NaN,8.97,2.0,NaN,5-6 hours,Healthy,B.Pharm,Yes,3.0,1.0,No,1
3,Yuvraj,Male,22.0,Mumbai,Working Professional,Teacher,NaN,5.0,NaN,NaN,1.0,Less than 5 hours,Moderate,BBA,Yes,10.0,1.0,Yes,1
4,Rhea,Female,30.0,Kanpur,Working Professional,Business Analyst,NaN,1.0,NaN,NaN,1.0,5-6 hours,Unhealthy,BBA,Yes,9.0,4.0,Yes,0


In [10]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 140700 entries, 0 to 140699
Data columns (total 19 columns):
 #   Column                                 Non-Null Count   Dtype  
---  ------                                 --------------   -----  
 0   Name                                   140700 non-null  object 
 1   Gender                                 140700 non-null  object 
 2   Age                                    140700 non-null  float64
 3   City                                   140700 non-null  object 
 4   Working Professional or Student        140700 non-null  object 
 5   Profession                             104070 non-null  object 
 6   Academic Pressure                      27897 non-null   float64
 7   Work Pressure                          112782 non-null  float64
 8   CGPA                                   27898 non-null   float64
 9   Study Satisfaction                     27897 non-null   float64
 10  Job Satisfaction                       112790 non-null  

In [11]:
df_train.isna().sum()

,0
Name,0
Gender,0
Age,0
City,0
Working Professional or Student,0
Profession,36630
Academic Pressure,112803
Work Pressure,27918
CGPA,112802
Study Satisfaction,112803


In [12]:
df_test.isna().sum()

,0
Name,0
Gender,0
Age,0
City,0
Working Professional or Student,0
Profession,24632
Academic Pressure,75033
Work Pressure,18778
CGPA,75034
Study Satisfaction,75033


In [13]:
df_train[df_train['Working Professional or Student']=='Student']

,Name,Gender,Age,City,Working Professional or Student,Profession,Academic Pressure,Work Pressure,CGPA,Study Satisfaction,Job Satisfaction,Sleep Duration,Dietary Habits,Degree,Have you ever had suicidal thoughts ?,Work/Study Hours,Financial Stress,Family History of Mental Illness,Depression
2,Yuvraj,Male,33.0,Visakhapatnam,Student,NaN,5.0,NaN,8.97,2.0,NaN,5-6 hours,Healthy,B.Pharm,Yes,3.0,1.0,No,1
8,Aishwarya,Female,24.0,Bangalore,Student,NaN,2.0,NaN,5.90,5.0,NaN,5-6 hours,Moderate,BSc,No,3.0,2.0,Yes,0
26,Aditya,Male,31.0,Srinagar,Student,NaN,3.0,NaN,7.03,5.0,NaN,Less than 5 hours,Healthy,BA,No,9.0,1.0,Yes,0
30,Prisha,Female,28.0,Varanasi,Student,NaN,3.0,NaN,5.59,2.0,NaN,7-8 hours,Moderate,BCA,Yes,4.0,5.0,Yes,1
32,Chhavi,Female,25.0,Jaipur,Student,NaN,4.0,NaN,8.13,3.0,NaN,5-6 hours,Moderate,M.Tech,Yes,1.0,1.0,No,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
140685,Aaradhya,Female,27.0,Surat,Student,NaN,5.0,NaN,5.75,5.0,NaN,5-6 hours,Unhealthy,Class 12,Yes,7.0,1.0,Yes,0
140686,Rohan,Male,27.0,Ludhiana,Student,NaN,2.0,NaN,9.40,3.0,NaN,Less than 5 hours,Healthy,MSc,No,0.0,3.0,Yes,0
140689,Ayaan,Male,31.0,Faridabad,Student,NaN,3.0,NaN,6.61,4.0,NaN,5-6 hours,Unhealthy,MD,No,12.0,2.0,No,0
140690,Rashi,Female,18.0,Ludhiana,Student,NaN,5.0,NaN,6.88,2.0,NaN,Less than 5 hours,Healthy,Class 12,Yes,10.0,5.0,No,1


In [14]:
df_train[df_train['Working Professional or Student']=='Student']['Profession'].unique()

array([nan, 'Civil Engineer', 'Architect', 'UX/UI Designer',
       'Digital Marketer', 'Content Writer', 'Educational Consultant',
       'Teacher', 'Student', 'Manager', 'Chef', 'Doctor', 'Lawyer',
       'Entrepreneur', 'Pharmacist'], dtype=object)

In [15]:
df_train.nunique()

,0
Name,422
Gender,2
Age,43
City,98
Working Professional or Student,2
Profession,64
Academic Pressure,5
Work Pressure,5
CGPA,331
Study Satisfaction,5


In [16]:
df_extra.head()

,Name,Gender,Age,City,Working Professional or Student,Profession,Academic Pressure,Work Pressure,CGPA,Study Satisfaction,Job Satisfaction,Sleep Duration,Dietary Habits,Degree,Have you ever had suicidal thoughts ?,Work/Study Hours,Financial Stress,Family History of Mental Illness,Depression
0,Pooja,Female,37,Ghaziabad,Working Professional,Teacher,NaN,2.0,NaN,NaN,4.0,7-8 hours,Moderate,MA,No,6,2,No,No
1,Reyansh,Male,60,Kalyan,Working Professional,Financial Analyst,NaN,4.0,NaN,NaN,3.0,5-6 hours,Unhealthy,B.Com,Yes,0,4,Yes,No
2,Manvi,Female,42,Bhopal,Working Professional,Teacher,NaN,2.0,NaN,NaN,3.0,5-6 hours,Moderate,M.Com,No,0,2,No,No
3,Isha,Female,44,Thane,Working Professional,Teacher,NaN,3.0,NaN,NaN,5.0,7-8 hours,Healthy,MD,Yes,1,2,Yes,No
4,Aarav,Male,48,Indore,Working Professional,UX/UI Designer,NaN,4.0,NaN,NaN,3.0,7-8 hours,Moderate,BE,Yes,6,5,Yes,No


In [17]:
df_extra.nunique()

,0
Name,216
Gender,2
Age,43
City,30
Working Professional or Student,2
Profession,35
Academic Pressure,5
Work Pressure,5
CGPA,312
Study Satisfaction,5


In [ ]:
num_features =[]
cat_features =[]
target_feature = 'Depression'

In [18]:
df_train['Degree'].unique()

array(['BHM', 'LLB', 'B.Pharm', 'BBA', 'MCA', 'MD', 'BSc', 'ME', 'B.Arch',
       'BCA', 'BE', 'MA', 'B.Ed', 'B.Com', 'MBA', 'M.Com', 'MHM', 'BA',
       'Class 12', 'M.Tech', 'PhD', 'M.Ed', 'MSc', 'B.Tech', 'LLM',
       'MBBS', 'M.Pharm', 'UX/UI Designer', 'MPA', 'BH', 'Nalini', 'BEd',
       'B.Sc', 'Veda', 'Bhopal', 'S.Tech', 'Degree', '20', 'Class 11',
       'H_Pharm', 'M', 'P.Com', 'BPharm', 'Business Analyst', 'M.Arch',
       'LL.Com', 'Data Scientist', 'MPharm', 'L.Ed', 'P.Pharm', 'Kalyan',
       'Unite', 'BArch', 'HR Manager', 'Badhya', 'S.Pharm', 'LLBA',
       'Vrinda', 'M. Business Analyst', 'Bhavesh', '0', 'LLCom', '29',
       'MTech', 'Vivaan', 'BPA', 'Plumber', '5.61', 'Brit', 'B.03',
       'Ritik', '5.56', 'MEd', 'B', 'B BA', '7.06', 'B.B.Arch', 'ACA',
       'Brithika', 'CGPA', '24', 'M_Tech', 'Pihu', 'BB', 'Jhanvi',
       'LLTech', 'Aarav', 'Entrepreneur', '8.56', 'LHM', 'Lata', 'S.Arch',
       'Marsh', 'HCA', '5.88', 'B.Student', 'LL B.Ed', 'M.S', 'Navya',
   

In [19]:
df_extra['Degree'].unique()

array(['MA', 'B.Com', 'M.Com', 'MD', 'BE', 'MCA', 'BA', 'LLM', 'BCA',
       'Class 12', 'B.Ed', 'M.Tech', 'LLB', 'B.Arch', 'ME', 'MBA',
       'M.Pharm', 'MBBS', 'PhD', 'BSc', 'MSc', 'MHM', 'BBA', 'BHM',
       'B.Tech', 'M.Ed', 'B.Pharm'], dtype=object)

In [20]:
for i in df_test.columns:
    z = set(df_test[i].unique()) - set(df_train[i].unique())
    print(i,len(z),list(z)[:10])

Name 132 ['Harshot', 'Rya', 'Vihaly', 'R. Aakash', 'Vivar', 'Avaan', 'Rudhi', 'Aamir', 'Vihirar', 'Abha']
Gender 0 []
Age 1 [32.08]
City 26 ['Avni', 'Unaly', 'Ghopal', 'Saurav', 'Golkata', 'Siddhesh', 'No.12', 'Pratham', 'Abhinav', 'Rolkata']
Working Professional or Student 0 []
Profession 17 ['Simran', 'MD', 'M.Pharm', 'MCA', '24th', 'B.Pharm', '3M', 'No', 'Samar', 'ME']
Academic Pressure 1 [nan]
Work Pressure 1 [nan]
CGPA 8 [5.92, 6.14, 5.5, 6.72, 5.36, 9.08, 6.43, nan]
Study Satisfaction 1 [nan]
Job Satisfaction 1 [nan]
Sleep Duration 10 ['9-10 hours', '0', '50-75 hours', '6 hours', 'Meerut', '60-65 hours', 'Have_you_ever_had_suicidal_thoughts', 'Vivan', '8-89 hours', '20-21 hours']
Dietary Habits 13 ['Mealy', 'MCA', 'Kolkata', 'Soham', 'Vivaan', '5 Healthy', 'Prachi', '5 Unhealthy', 'Educational', 'Academic']
Degree 41 ['3.0', 'Banchal', 'B.M.Com', 'BHCA', 'Rupak', 'BTech', '5.65', 'Travel Consultant', 'Pune', 'Vibha']
Have you ever had suicidal thoughts ? 0 []
Work/Study Hours 0 [

In [21]:
df_train['Academic Pressure'].unique()

array([nan,  5.,  2.,  3.,  4.,  1.])

In [25]:
for i in range(len(df_train)):
    if df_train.loc[i]['Working Professional or Student']=='Student':
        df_train.loc[i,'Pressure'] = df_train.iloc[i]['Academic Pressure']
    else:
        df_train.loc[i,'Pressure'] = df_train.iloc[i]['Work Pressure']

In [51]:
df_train.head()

,Name,Gender,Age,City,Working Professional or Student,Profession,Academic Pressure,Work Pressure,CGPA,Study Satisfaction,Job Satisfaction,Sleep Duration,Dietary Habits,Degree,Have you ever had suicidal thoughts ?,Work/Study Hours,Financial Stress,Family History of Mental Illness,Depression,Pressure,Satisfaction
0,Aaradhya,Female,49.0,Ludhiana,Working Professional,Chef,NaN,5.0,NaN,NaN,2.0,More than 8 hours,Healthy,BHM,No,1.0,2.0,No,0,5.0,2.0
1,Vivan,Male,26.0,Varanasi,Working Professional,Teacher,NaN,4.0,NaN,NaN,3.0,Less than 5 hours,Unhealthy,LLB,Yes,7.0,3.0,No,1,4.0,3.0
2,Yuvraj,Male,33.0,Visakhapatnam,Student,NaN,5.0,NaN,8.97,2.0,NaN,5-6 hours,Healthy,B.Pharm,Yes,3.0,1.0,No,1,5.0,5.0
3,Yuvraj,Male,22.0,Mumbai,Working Professional,Teacher,NaN,5.0,NaN,NaN,1.0,Less than 5 hours,Moderate,BBA,Yes,10.0,1.0,Yes,1,5.0,1.0
4,Rhea,Female,30.0,Kanpur,Working Professional,Business Analyst,NaN,1.0,NaN,NaN,1.0,5-6 hours,Unhealthy,BBA,Yes,9.0,4.0,Yes,0,1.0,1.0


In [52]:
# Calculate
df_train[(df_train['Work Pressure'].isna())&(df_train['Academic Pressure'].isna())]
# df_train[(~df_train['Work Pressure'].isna())&(~df_train['Academic Pressure'].isna())]
# df_train[(df_train['Study Satisfaction'].isna())&(df_train['Job Satisfaction'].isna())]
# df_train[(~df_train['Study Satisfaction'].isna())&(~df_train['Job Satisfaction'].isna())]

# df_test[(~df_test['Study Satisfaction'].isna())&(~df_test['Job Satisfaction'].isna())]
# df_test[(~df_test['Work Pressure'].isna())&(~df_test['Academic Pressure'].isna())]

,Name,Gender,Age,City,Working Professional or Student,Profession,Academic Pressure,Work Pressure,CGPA,Study Satisfaction,Job Satisfaction,Sleep Duration,Dietary Habits,Degree,Have you ever had suicidal thoughts ?,Work/Study Hours,Financial Stress,Family History of Mental Illness,Depression,Pressure,Satisfaction
15846,Shrey,Male,53.0,Vasai-Virar,Working Professional,NaN,NaN,NaN,NaN,NaN,5.0,More than 8 hours,Unhealthy,Class 12,No,12.0,1.0,No,0,3.0,5.0
17549,Tanya,Female,20.0,Patna,Student,NaN,NaN,NaN,5.55,NaN,NaN,5-6 hours,Moderate,Class 12,No,0.0,3.0,Yes,0,3.0,3.0
34694,Kriti,Female,46.0,Meerut,Working Professional,NaN,NaN,NaN,NaN,NaN,NaN,5-6 hours,Moderate,Class 12,Yes,12.0,3.0,No,0,3.0,3.0
35612,Siddhesh,Male,27.0,Vadodara,Working Professional,NaN,NaN,NaN,NaN,NaN,NaN,Less than 5 hours,Unhealthy,B.Pharm,No,11.0,3.0,No,1,3.0,3.0
57196,Manvi,Female,20.0,Visakhapatnam,Working Professional,NaN,NaN,NaN,NaN,NaN,NaN,Less than 5 hours,Moderate,Class 12,No,0.0,4.0,Yes,1,3.0,3.0
60348,Nishant,Male,29.0,Agra,Working Professional,NaN,NaN,NaN,9.72,2.0,NaN,5-6 hours,Moderate,B.Ed,No,7.0,2.0,Yes,0,3.0,3.0
70453,Veda,Female,20.0,Ahmedabad,Student,NaN,NaN,NaN,NaN,NaN,2.0,Less than 5 hours,Moderate,Class 12,Yes,12.0,3.0,Yes,1,3.0,3.0
76104,Vanya,Female,20.0,Nashik,Working Professional,NaN,NaN,NaN,NaN,NaN,1.0,7-8 hours,Moderate,Class 12,No,11.0,1.0,No,1,3.0,1.0
85126,Monika,Female,27.0,Kanpur,Working Professional,Mechanical Engineer,NaN,NaN,NaN,NaN,5.0,5-6 hours,Healthy,BE,Yes,10.0,5.0,No,1,3.0,5.0
86606,Tushar,Male,18.0,Thane,Working Professional,NaN,NaN,NaN,NaN,NaN,1.0,7-8 hours,Moderate,Class 12,Yes,10.0,3.0,No,1,3.0,1.0


In [1]:
df_extra.head(20)

NameError: name 'df_extra' is not defined

In [ ]:
# For train data
condition = df_train['Working Professional or Student']=='Student'
median_academic_pressure = df_train['Academic Pressure'].median()
median_work_pressure = df_train['Work Pressure'].median()
median_study_satisfaction = df_train['Study Satisfaction'].median()
median_job_satisfaction = df_train['Job Satisfaction'].median()

df_train.loc[condition,'Pressure'] = df_train['Academic Pressure'].fillna(median_academic_pressure)
df_train.loc[condition,'Satisfaction'] = df_train['Academic Pressure'].fillna(median_study_satisfaction)

df_train.loc[~condition,'Pressure'] = df_train['Work Pressure'].fillna(median_work_pressure)
df_train.loc[~condition,'Satisfaction'] = df_train['Job Satisfaction'].fillna(median_job_satisfaction)

# For test data
condition = df_test['Working Professional or Student']=='Student'

df_test.loc[condition,'Pressure'] = df_test['Academic Pressure'].fillna(median_academic_pressure)
df_test.loc[condition,'Satisfaction'] = df_test['Academic Pressure'].fillna(median_study_satisfaction)

df_test.loc[~condition,'Pressure'] = df_test['Work Pressure'].fillna(median_work_pressure)
df_test.loc[~condition,'Satisfaction'] = df_test['Job Satisfaction'].fillna(median_job_satisfaction)

# For extra data
condition = df_extra['Working Professional or Student']=='Student'

df_extra.loc[condition,'Pressure'] = df_extra['Academic Pressure'].fillna(median_academic_pressure)
df_extra.loc[condition,'Satisfaction'] = df_extra['Academic Pressure'].fillna(median_study_satisfaction)

df_extra.loc[~condition,'Pressure'] = df_extra['Work Pressure'].fillna(median_work_pressure)
df_extra.loc[~condition,'Satisfaction'] = df_extra['Job Satisfaction'].fillna(median_job_satisfaction)



In [13]:
median_academic_pressure = df_train['Academic Pressure'].median()
median_work_pressure = df_train['Work Pressure'].median()
median_study_satisfaction = df_train['Study Satisfaction'].median()
median_job_satisfaction = df_train['Job Satisfaction'].median()
# Write a common function
def transform_data(df):
    condition = df['Working Professional or Student']=='Student'

    df.loc[condition,'Profession'] = 'Student'
    df.loc[condition,'Pressure'] = df['Academic Pressure'].fillna(median_academic_pressure)
    df.loc[condition,'Satisfaction'] = df['Academic Pressure'].fillna(median_study_satisfaction)
    df.loc[~condition,'Pressure'] = df['Work Pressure'].fillna(median_work_pressure)
    df.loc[~condition,'Satisfaction'] = df['Job Satisfaction'].fillna(median_job_satisfaction)
    return df

df_train = transform_data(df_train)
df_test = transform_data(df_test)
df_extra = transform_data(df_extra)

### Points to be remembered:
1. Degree column has some inconsistent data.
1.

In [ ]:
features = df_test.columns


def visualization(dataframe, features=features):
    summary = {
        'Columns'   :   [],
        'Count'     :   [],
        'Unique count': [],
        'Max'       :   [],
        'Min'       :   [],
        'Mean'      :   [],
        'Std'       :   [],
        '5%'        :   [],
        '25%'        :   [],
        '50%'        :   [],
        '75%'        :   [],
        '95%'        :   [],
        # IQR, Low Bound, High Bound
        'IQR'       :   [],
        'Low Bound' :   [],
        'High Bound':   [],
        '< Low Bound' :   [],
        '> High Bound':   [],
        'Outliers'  : [],
        '% Outliers'  : [],
    }
    for col in features:
        summary['Columns'].append(col)
        summary['Count'].append(dataframe[col].notnull().sum())
        summary['Unique count'].append(dataframe[col].nunique())
        summary['Max'].append(dataframe[col].max())
        summary['Min'].append(dataframe[col].min())
        summary['Mean'].append(dataframe[col].mean())
        summary['Std'].append(dataframe[col].std())
        for num in [5, 25, 50, 75, 95]:
            summary[f"{num}%"].append(np.percentile(dataframe[col], num))
        summary['IQR'].append(summary['75%'][-1] - summary['25%'][-1])
        summary['Low Bound'].append(summary['50%'][-1]-1.5*summary['IQR'][-1])
        summary['High Bound'].append(summary['50%'][-1]+1.5*summary['IQR'][-1])

        summary["< Low Bound"].append(len(dataframe[dataframe[col]<summary["Low Bound"][-1]]))
        summary["> High Bound"].append(len(dataframe[dataframe[col]>summary["High Bound"][-1]]))
        summary["Outliers"].append(summary["< Low Bound"][-1]+ summary["> High Bound"][-1])
        summary["% Outliers"].append(summary["Outliers"][-1]/len(dataframe)*100.0)
    summary = pd.DataFrame(summary)

    return summary

def missing_values_table(dataframe):
    x = dataframe.isna().sum()/len(dataframe)*100
    x = x.reset_index()
    x.columns = ['feature', 'percentage-of-null']
    x = x.sort_values(by='percentage-of-null', ascending=False).reset_index(drop=True)
    x['unique'] = [dataframe[i].nunique() for i in x['feature']]
    x['unique-vals'] = [",".join (sorted(map(str,dataframe[i].unique()))) for i in x['feature']]
    # return x[x['percentage-of-null']>0]
    return x

# Data Visualization and Analysis
# Numerical and Categorical visualization
def visualize_categorical_distributions(df):
    """
    Visualizes the distribution of categorical features in the DataFrame.

    Parameters:
    - df (pd.DataFrame): The DataFrame containing the categorical features to visualize.

    Returns:
    - None: Displays the plots.
    """
    # Identify categorical columns
    categorical_columns = df.select_dtypes(include=['object']).columns

    # Set up the figure for multiple subplots
    num_cols = 3  # Number of columns for the subplot grid
    num_rows = (len(categorical_columns) + num_cols - 1) // num_cols  # Calculate number of rows needed

    fig, axes = plt.subplots(num_rows, num_cols, figsize=(15, num_rows * 5))
    fig.suptitle('Distribution of Categorical Features', fontsize=16)

    # Flatten axes array for easy iteration
    axes = axes.flatten()

    # Iterate over each categorical column and create a bar plot
    for i, col in enumerate(categorical_columns):
        sns.barplot(
            x=df[col].value_counts().index,
            y=df[col].value_counts().values,
            ax=axes[i],
            palette="viridis"
        )
        # Setting titles and labels
        axes[i].set_title(f'Distribution of {col}', fontsize=14)
        axes[i].set_xlabel(col, fontsize=12)
        axes[i].set_ylabel('Count', fontsize=12)
        axes[i].tick_params(axis='x')

    # Remove unused axes
    for j in range(i + 1, len(axes)):
        fig.delaxes(axes[j])

    # Adjust layout
    plt.tight_layout(rect=[0, 0.03, 1, 0.95])  # Adjust the main title space
    plt.show()

def visualize_numerical_distributions(df, exclude_columns='id'):
    """
    Visualizes the distribution of numerical features in the DataFrame using histograms and KDE plots,
    excluding specified columns.

    Parameters:
    - df (pd.DataFrame): The DataFrame containing the numerical features to visualize.
    - exclude_columns (list): List of column names to exclude from visualization.

    Returns:
    - None: Displays the plots.
    """
    # Ensure exclude_columns is a list
    if exclude_columns is None:
        exclude_columns = []

    # Identify numerical columns and exclude specified ones
    numerical_columns = df.select_dtypes(include=['number']).columns
    numerical_columns = [col for col in numerical_columns if col not in exclude_columns]

    # Set up the figure for multiple subplots
    num_cols = 3  # Number of columns for the subplot grid
    num_rows = (len(numerical_columns) + num_cols - 1) // num_cols  # Calculate number of rows needed

    fig, axes = plt.subplots(num_rows, num_cols, figsize=(18, 5 * num_rows))
    fig.suptitle('Distribution of Numerical Features', fontsize=16)

    # Flatten axes array for easy iteration
    axes = axes.flatten()

    # Iterate over each numerical column and create a histogram with KDE
    for i, col in enumerate(numerical_columns):
        sns.histplot(df[col], kde=True, ax=axes[i], color="skyblue", element="step", stat="density")
        axes[i].set_title(f'Distribution of {col}', fontsize=14)
        axes[i].set_xlabel(col, fontsize=12)
        axes[i].set_ylabel('Density', fontsize=12)

    # Remove unused axes
    for j in range(i + 1, len(axes)):
        fig.delaxes(axes[j])

    # Adjust layout
    plt.tight_layout(rect=[0, 0.03, 1, 0.95])  # Adjust the main title space
    plt.show()


In [ ]:
# Optional
def preprocess_categorical_data(df, df_t):
    """
    Preprocesses the DataFrame by replacing non-alphabetical characters in categorical columns
    and imputing missing values in categorical columns using mode and in numeric columns using median.

    Parameters:
    - df (pd.DataFrame): The input DataFrame.

    Returns:
    - pd.DataFrame: The processed DataFrame.
    """
    def replace_non_alphabetical(value):
        """Replaces non-alphabetical values with NaN if they are not a single character."""
        if isinstance(value, str) and (len(value) == 1 and value.isalpha()):
            return value
        return np.nan

    # Identify categorical and numeric columns
    categorical_columns = df_t.select_dtypes(include=['object']).columns
    numeric_columns = df_t.select_dtypes(include=[np.number]).columns

    # Replace non-alphabetical values with NaN in categorical columns
    for col in categorical_columns:
        df[col] = df[col].apply(replace_non_alphabetical)
        df_t[col] = df_t[col].apply(replace_non_alphabetical)

    # Impute missing values in categorical columns using mode
    for col in categorical_columns:
        if df[col].isnull().any():
            mode_value = df[col].mode()[0] if not df[col].mode().empty else 'Unknown'  # Safe mode handling
            df[col].fillna(mode_value, inplace=True)
            df_t[col].fillna(mode_value, inplace=True)

    # Impute missing values in numeric columns using median
    for col in numeric_columns:
        median_value = df[col].median()
        if df[col].isnull().any():
            df[col].fillna(median_value, inplace=True)
        df_t[col].fillna(median_value, inplace=True)

    return df, df_t


# Apply the functions to train and test data
print("Imputing noise with mode for categoricals and median for numericals...")
train, test = preprocess_categorical_data(df_train, df_test)


# Print summary of missing values after processing
print("\nMissing values in train_data after processing: ", train.isnull().sum().sum())
print("\nMissing values in test_data after processing: ", test.isnull().sum().sum())


In [ ]:
def categorize_low_frequency_values(df_train, df_test, threshold=100):
    """
    Replaces categories with less than a given threshold in all categorical columns
    with the category 'Other', applying the same transformation to both training and test datasets.

    Parameters:
    - df_train (pd.DataFrame): The training DataFrame.
    - df_test (pd.DataFrame): The test DataFrame.
    - threshold (int): Frequency threshold below which categories are replaced with 'Other'.

    Returns:
    - df_train (pd.DataFrame): Modified training DataFrame.
    - df_test (pd.DataFrame): Modified test DataFrame.
    - mapping_dict (dict): Dictionary containing the mappings of replaced values for each column.
    """
    mapping_dict = {}

    # Identify categorical columns
    categorical_columns = df_train.select_dtypes(include=['object']).columns

    for col in categorical_columns:
        # Find values to replace
        value_counts = df_train[col].value_counts()
        values_to_replace = value_counts[value_counts < threshold].index

        # Determine the mode of the column
        mode_value = df_train[col].mode()[0]

        # Create mapping for the current column if there are values to replace
        if len(values_to_replace) > 0:
            # Store the mapping of original values to the mode
            mapping_dict[col] = {value: mode_value for value in values_to_replace}

            # Replace in training and test data
            df_train[col] = df_train[col].replace(values_to_replace, mode_value)
            df_test[col] = df_test[col].replace(values_to_replace, mode_value)

    return df_train, df_test, mapping_dict

train, test, mappings = categorize_low_frequency_values(train, test)

In [ ]:
from scipy.stats import boxcox
def handle_skewness(df, df_t, threshold=1.0):
    """
    Applies Box-Cox transformation to numerical columns in the DataFrame where skewness exceeds a threshold.

    Parameters:
    - df (pd.DataFrame): The input DataFrame.
    - threshold (float): Skewness threshold to decide which columns to transform.

    Returns:
    - pd.DataFrame: DataFrame with transformed columns.
    - dict: Dictionary of lambda values used for Box-Cox transformation for each column.
    """
    numeric_cols = df.select_dtypes(include=['number']).columns

    for col in numeric_cols:
        skewness = df[col].skew()
        # Check the skewness and ensure positive values for Box-Cox
        if skewness > threshold:
            # Adding 1 to shift all data to positive if there are zero or negative values
            df[col] = df[col] + 1
            df[col], fitted_lambda = boxcox(df[col])
            df_t[col] = df_t[col] + 1
            df_t[col] = boxcox(df_t[col], lmbda=fitted_lambda)


    return df, df_t

# Example usage:
# df is your DataFrame containing the numerical data
train, test = handle_skewness(train, test)


In [ ]:
def visualize_box_plots(df):
    """
    Visualizes the distribution of numerical features in the DataFrame using box plots to identify outliers.

    Parameters:
    - df (pd.DataFrame): The DataFrame containing the numerical features to visualize.

    Returns:
    - None: Displays the box plots.
    """
    # Identify numerical columns
    numerical_columns = df.select_dtypes(include=['number']).columns

    # Set up the figure for multiple subplots
    num_cols = 3  # Number of columns for the subplot grid
    num_rows = (len(numerical_columns) + num_cols - 1) // num_cols  # Calculate number of rows needed

    fig, axes = plt.subplots(num_rows, num_cols, figsize=(18, 5 * num_rows))
    fig.suptitle('Box Plot of Numerical Features', fontsize=16)

    # Flatten axes array for easy iteration
    axes = axes.flatten()

    # Iterate over each numerical column and create a box plot
    for i, col in enumerate(numerical_columns):
        sns.boxplot(x=df[col], ax=axes[i], color="skyblue")
        axes[i].set_title(f'Box Plot of {col}', fontsize=14)
        axes[i].set_xlabel(col, fontsize=12)

    # Remove unused axes
    for j in range(i + 1, len(axes)):
        fig.delaxes(axes[j])

    # Adjust layout
    plt.tight_layout(rect=[0, 0.03, 1, 0.95])  # Adjust the main title space
    plt.show()

visualize_box_plots(train)

In [ ]:
def calculate_outliers_percentage(df):
    """
    Calculates the percentage of data considered outliers based on the IQR method for each numerical column.

    Parameters:
    - df (pd.DataFrame): DataFrame to analyze.

    Returns:
    - None: Prints the percentage of outliers for each numerical column.
    """
    outlier_counts = {}
    for column in df.select_dtypes(include=['number']).columns:
        Q1 = df[column].quantile(0.25)
        Q3 = df[column].quantile(0.75)
        IQR = Q3 - Q1
        lower_bound = Q1 - 1.5 * IQR
        upper_bound = Q3 + 1.5 * IQR

        # Calculate outliers
        outliers = df[(df[column] < lower_bound) | (df[column] > upper_bound)]
        outlier_counts[column] = len(outliers)

    # Print the percentage of outliers for each column
    for column in outlier_counts:
        percentage = (outlier_counts[column] / len(df)) * 100
        print(f"Percentage of outliers in {column}: {percentage:.2f}%")

# Example usage:
calculate_outliers_percentage(train)

In [ ]:
def handle_outliers(df,df_t):
    """
    Handles outliers in a DataFrame by capping based on the IQR method.

    Parameters:
    - df (pd.DataFrame): DataFrame to process.

    Returns:
    - pd.DataFrame: DataFrame with outliers handled.
    """
    for column in ['cap-diameter', 'stem-height', 'stem-width']:
        Q1 = df[column].quantile(0.25)
        Q3 = df[column].quantile(0.75)
        IQR = Q3 - Q1
        lower_bound = Q1 - 1.5 * IQR
        upper_bound = Q3 + 1.5 * IQR

        # Capping
        df[column] = df[column].clip(lower=lower_bound, upper=upper_bound)
        df_t[column] = df_t[column].clip(lower=lower_bound, upper=upper_bound)

    return df, df_t

# Apply to both training and test datasets
train, test = handle_outliers(train, test)

In [ ]:
train.to_csv('train_processed.csv', index=False)
test.to_csv('test_processed.csv', index=False)

In [ ]:
with mlflow.start_run() as run:
    artifact_uri, run_id = run.info.artifact_uri, run.info.run_id
    mlflow.log_artifact("train_processed.csv")
    mlflow.log_artifact("test_processed.csv")